## Create Offspring Branch

In [ ]:
import os 
os.chdir('..')

In [ ]:
from utils.rpc.lib import json_rpc_call
from utils.encode.bytes import encode_bytes_to_base64_str, decode_base64_str_to_bytes
from config.branch_cfg import TWIG_BRANCH_TYPE_ID
from config.bucket_cfg import BUCKET_ID_TYPE_NO_REF, BUCKET_ID_TYPE_WITH_ID_REF, DEFAULT_MOLECULAR_BUCKET_SCHEMA, DEFAULT_ATOMIC_BUCKET_SCHEMA
BYTES0 = encode_bytes_to_base64_str(bytes(0))

#### Create Genesis Branch

In [ ]:
signature = BYTES0
accept_conflicts = False
msg = 'Genesis Submit'
name = 'Genesis Branch'
create_branch_kwargs = dict(branch_type=TWIG_BRANCH_TYPE_ID, name=name, signature=signature, accept_conflicts=accept_conflicts, msg=msg)

In [ ]:
genesis_response = json_rpc_call(method="create_genesis_branch", params=create_branch_kwargs)
# response is inside the result field
print("Test 'create_genesis_branch' passed with branch ID:", genesis_response['result'], "\nand base64 decoded branch ID:", decode_base64_str_to_bytes(genesis_response['result']))

#### Create Offspring Branch

In [ ]:
create_offspring_branch_kwargs = dict(
    parent_branch_id=genesis_response['result'], 
    branch_type=TWIG_BRANCH_TYPE_ID, 
    name='Offspring Branch', 
    signature=signature, 
    accept_conflicts=accept_conflicts, 
    msg='Offspring Submit')

In [ ]:
offspring_response = json_rpc_call(method="create_offspring_branch_at_head", params=create_offspring_branch_kwargs)
# response is inside the result field
print("Test 'create_offspring_branch_at_head' passed with branch ID:", offspring_response['result'], "\nand base64 decoded branch ID:", decode_base64_str_to_bytes(offspring_response['result']))

#### Create Offspring branch at particular submit

In [ ]:
rpc_kwargs = dict(branch_id=offspring_response["result"], deserialize_buckets=False) #rpc_genesis_branch_response["decoded_branch_id"], deserialize_buckets=True)
response = json_rpc_call(method="get_branch_data_from_branch_id", params=rpc_kwargs)

In [ ]:
genesis_submit_id = response["result"]["submit"]["parent_submit_id"] 
response["result"]

In [ ]:
create_another_offspring_branch_kwargs = dict(
    parent_branch_id=genesis_response['result'],
    parent_submit_id=genesis_submit_id,
    branch_type=TWIG_BRANCH_TYPE_ID,
    name='Another Offspring Branch',
    signature=signature,
    accept_conflicts=accept_conflicts,
    msg='Another Offspring Submit')

In [ ]:
another_offspring_response = json_rpc_call(method="create_offspring_branch_at_head", params=create_offspring_branch_kwargs)
# response is inside the result field
print("Test 'create_offspring_branch_at_head' passed with branch ID:", another_offspring_response['result'], "\nand base64 decoded branch ID:", decode_base64_str_to_bytes(another_offspring_response['result']))